# Annotations & Typing & Dataclasses
- Usecase
    - Better documentation
    - static type checking (e.g. with mypy)
- Annotations part of stdlib & __future__ module at the moment
    - further annotations will become part of stdlib in 3.10
    - Full support, e.g. for function return type only with future import
- Annotations
    - Only for documentation / linting
    - arguments / return types / variables (PEP526 Py3.6)
    - Gradual typing: only functions with type annotations are checked!
        - can introduce typing method by method, file by file, etc.
- Versions
    - Python3: Arguments e.g. def __init__(self, name: str) (before via comment)
    - Python3.5: Type Hints part of stdlib e.g. from typing import SupportsFloat
    - Python3.6: Instance variable annotations. e.g. name: str 
    - Python3.7:  __future__ annotations for postponing annotation evaluation (not at defintion time)
    - Pyton3.10: __future__ annotations -> stdlib (e.g. return type)
- Typing (PEP484)
    - Do's:
        - Annotate function parameters and return types
        - Annotate variables only where needed (can mostly be infered = guess by type checker)
        - Unions and Optionals only where needed
        - Return types of Unions/Optionals should be ommited -> better to annote with overload
        - Overload and Generics (e.g. TypeVar) teach type checker to be smarter
    - Any = means Any type is okey
    - Union = means one of the provided types (either A or B or ...)
    - SupportsFloat = example for abstracted data types that can be casted to float
    - List, Tuple etc. => for containers
    - NewType: Define own type -> E.g. Price and Quantity both a decimal, but they are not the same
        - If more complex new type with a lot of functions -> should create a new class
        - But for example define a NewType of Price that actually nothing more than a float.
        - Alias nams for data types + type checking with mypy
    - TypeVar: 
        - Compared with Union here compiler knows return type (see below)
        - Can be of type Any (unbounded) or limit the types (bounded)
        - E.g. AnyStr = TypeVar('AnyStr', Text, bytes) = bounded 
        - E.g. T = TypeVar('T') -> often used for generics = unbounded
        - Cannot be redefined
    - Optional = for Optional args (special version of Union[Foo, None])
    - Generic: Inherit from Generic keywork to make user defined class Generic!
    - Nominal vs structural typing
        - Structural typing: Describe capability not inheritence -> Interface c#
        - Nominal typing: Describe inheritence (class hierarchy)
- Can be nicely combined with Dataclasses or used separately
- Dataclasses
    - Generates __init__(), repr(), str() and other stuff based on Annotations
- MyPy uses annotations to verify code (static code analysis)
    - Run from cli with: mypy file.py or python -m mypy Snips/try.py 
- See:
    - https://docs.python.org/3/library/typing.html
    - https://www.youtube.com/watch?v=UQo-ebJk4a4
    - https://www.youtube.com/watch?v=pMgmKJyWKn8 (Thanks to Carl Meyer)
        - check out typing_extensions Pypi module (Video 18:00)
    - Hatches
        - Any
        - cast to lie to type checker 
        - ignore e.g. stacked decorators
        - stub (phi) for cython like c header files ()
        


In [53]:
from __future__ import annotations
from dataclasses import dataclass
from datetime import date
from typing import Optional, Dict

class Car:
    # Argument annotation and return value in 3.x
    def __init__(self, make: str, brand: str, year: int):
        self.make = make
        self.brand = brand
        self.year = year

    # typing module added in 3.5
    def age(self, different_year: Optional[int] = None) -> int:
        if different_year:
            return date.today().year - different_year   
        return date.today().year - self.year

@dataclass
class Person:
    # Variable annotation in 3.6
    name: str
    age: int


p1 = Person(name='Peter', age=52)
c1 = Car(make='Mustang', brand='Ford', year=1960)
print(p1.name)
print(f'{p1}') #str function
print(f'{p1!r}') #Repr function

print(c1.make)
print(f'{c1}') #str function
print(f'{c1!r}') #Repr function
print(c1.age())
print(c1.age(different_year=1959))

# Before py39 (typing module needed)
driver: Dict[Person, Car] = {}
driver[p1.name] = c1.brand
print(driver)

# py39+ (integrated, no need to import typing module)
driver: dict[Person, Car] = {}
driver[p1.name] = c1.brand
print(driver)

Peter
Person(name='Peter', age=52)
Person(name='Peter', age=52)
Mustang
60
61
{'Peter': 'Ford'}
{'Peter': 'Ford'}


In [31]:
# Annotating Vars and return values
def greeting(name: str) -> str:
    return 'Hello ' + name

greeting(name='Test')

'Hello Test'

In [41]:
# Typing 
from __future__ import annotations
from typing import Union, Any, Type, SupportsFloat
import numbers

Vector = list[float]

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]

new_vector = scale(2.0, [1.0, -4.2, 5.4])
print(new_vector)

# Two dimensional vector class
class Vector:
    def __init__(self, x: SupportsFloat = 0, y: SupportsFloat = 0):
        self.x = x
        self.y = y

    # Multiplication of vector can be with a scaler value (float) or with another Vector
    # Union: Return Vector or SupportsFloat
    # Union: Pass in Vector or SupportsFloat
    def __mul__(self, other: Union[Vector, SupportsFloat]) -> Union[Vector, SupportsFloat]:
        if isinstance(other, Vector):
            return self.x * other.x + self.y * other.y
        elif isinstance(other, numbers.Real):
            return Vector(self.x * other, self.y * other)
        else:
            raise TypeError('Must pass vector or int/float value')

    def __eq__(self, other: Any):
        return self.x == other.x and self.y == other.y

scaler = 3
v1 = Vector(x=2, y=2)
v2 = Vector(x=4, y=4)
v3 = Vector(x=2, y=2)

print(v1 == v2)
print(v1 == v3)

v3 *= 3 
print(v1*v2)
print(v3.x, v3.y)   


[2.0, -8.4, 10.8]
False
True
16
6 6


In [62]:
# Typing 2 - TypeVar, Generics
# Type variables exist primarily for the benefit of static type checkers

from typing import TypeVar, Generic

T = TypeVar('T')

class Stack(Generic[T]):
    def __init__(self) -> None:
        # Create an empty list with items of type T
        self.items: List[T] = []

    def push(self, item: T) -> None:
        self.items.append(item)

    def pop(self) -> T:
        return self.items.pop()

    def empty(self) -> bool:
        return not self.items

# Construct an empty Stack[int] instance
stack = Stack[int]()
stack.push(2)
stack.pop()
stack.push('x')        # Mypy Type error - remember: no runtime error
stack.push(5)        
stack.pop()
stack.pop()

'x'

In [63]:
T = TypeVar('T')  # Can be anything
A = TypeVar('A', str, bytes)  # Must be str or bytes

In [64]:
# Typing 3 - Type inference (guessing)
# Do's:
#   - Annotate Function Signatures
#   - Annotate Variables (only if you have to -> type checker can mostly find out himself)

from typing import Tuple

class Photo:
    def __init__(self, width: int, height: int) -> None:
        self.width = width
        self.height = height
        # self.tags = []  # Empty Container type checker would fail
        self.tags: List[str] = []   # Python 3.6

    # type checker complains because it can infere that those are int
    # def get_dimensions(self) -> Tuple[str, str]:    
    def get_dimensions(self) -> Tuple[int, int]:
        return (self.width, self.height)

p1 = Photo(10, 10)
print(p1.get_dimensions())


(10, 10)


In [65]:
# Typing 4 - Union / Optional / Overloads
# Union
# Optional
#   - Omit Optional or Union as return type because type checker could fail
# Overload
#   - In case Union as return type would be used, better to overload
from typing import Union, Optional, overload

def get_foo_or_bar(id: int) -> Union[Foo, Bar]:
    pass

def get_foo_or_none(id: int) -> Union[Foo, None]:
    pass

# Special form of the one before
# Equal to Union[Foo, None]
def get_foo_or_none(id: int) -> Optional[Foo]:
    pass

# Better for the type checker
# Not used at runtime if annotated with overload
# Only for the type checker
# -> Reason why pass is okey, no function body needed!
# Without the overload methods, type checker would throw:
#   error: NoneType has no attribute id when calling xy_foo.id
@overload
def get_foo(foo_id: None) -> None:
    pass

@overload
def get_foo(foo_id: int) -> int:
    pass

def get_foo(foo_id: Optional[int]) -> Optional[Foo]:
    if foo_id is None:
        return None
    return Foo(foo_id)

In [75]:
# Typing 5 - Generic functions
# TypeVar = Placeholder for a type

# Must be str or bytes -> Value restriction
# Name = TypeVar('Name') -> Any type possible (unbounded)
# Name = TypeVar('Name', type1, type2, type3) -> only of type1,2,3 (bounded)
from typing import TypeVar

AnyStr = TypeVar('AnyStr', str, bytes)

def concat(a: AnyStr, b:AnyStr) -> AnyStr:
    return a + b

print(concat('Helo', 'World'))
print(concat(b'foo', b'bar'))

# Uncomment and run python -m mypy Snips/try.py   
# Compiler knows return type is string
# reveal_type(concat('foo', 'bar'))

# Compiler knows return type is bytes
# reveal_type(concat(b'foo', b'bar'))
# -> big advantage over Union

# print(concat('Helo', b'bytes'))     # Compiler complains and does not work at runtime!
print(concat(10, 10))               # Works at runtime but type checker complains!



HeloWorld
b'foobar'
20


In [77]:
# Typing 6 - NewType
# Define own type to describe something
# More sophisticated stuff should be done with a new class!
from typing import NewType
from decimal import *

# Alias but not type checked
# Price = Decimal

# Alias + type checkign from mypy
Name = NewType('Name', str)
Price = NewType('Price', Decimal)
Quantity = NewType('Quantity', Decimal)

# p = Price(Decimal('1.4'))
# reveal_type(price)    # module.Price

# Old style
# def place_order(price: Decimal, quantity: Decimal) -> None:
#    ...

# Cannot pass a Decimal -> mypy complains even though under the hood a Decimal
# Cannot pass a Quantity -> mypy complains even though under the hood a Decimal
def f(price: Price) -> None:
    pass

def place_order(price: Price, quantity: Quantity) -> None:
    ...

In [85]:
# Typing 7 - Generic
from typing import TypeVar, Generic

# TypeVar can be of any type = unbounded = no restrictions
# Otherwise TypeVar would need to be defined like TypeVar('T', float, int) for example
T = TypeVar('T')

# By inheriting from Generic own classes can be made generic
class LoggedVar(Generic[T]):
    def __init__(self, value: T, name: str) -> None:
        self.name = name
        self.value = value

    def set(self, new: T) -> None:
        self.log('Set ' + repr(self.value))
        self.value = new

    def get(self) -> T:
        self.log('Get ' + repr(self.value))
        return self.value

    def log(self, message: str) -> None:
        print('{}: {}'.format(self.name, message))

int_logger = LoggedVar[int](value=5, name='Integer Logger')
int_logger.set(new=10)
int_logger.set(new=7)
int_logger.set(new='halo')  #mypy complains
int_logger.set(new=8)

str_logger = LoggedVar[str](value='helo', name='Integer Logger')
str_logger.set(new=10)  #mypy complains
str_logger.set(new=7)   #mypy complains
str_logger.set(new='halo')
str_logger.set(new=8)   #mypy complains


Integer Logger: Set 5
Integer Logger: Set 10
Integer Logger: Set 7
Integer Logger: Set 'halo'


In [87]:
# Typing 8 - "Full" example
# Define own Generic Series class
# Define overloads because __getitem__ ([]-operator) can return a single value or a sequence of values
#   ... (no method body needed) and @overload 

from typing import Generic, overload, Sequence, TypeVar, Union

ValueType = TypeVar('ValueType')

class Series(Generic[ValueType]):
    def __init__(self, data: Sequence[ValueType]):
        self._data = data

    @overload
    def __getitem__(self, index: int) -> ValueType:
        ...
    
    @overload
    def __getitem__(self, index: slice) -> Sequence[ValueType]:
        ...

    def __getitem__(self, index: Union[int, slice]) -> Union[ValueType, Sequence[ValueType]]:
        return self._data[index]

s = Series[int]([2, 4, 6, -1, 3])
s[0] + 5
sum(s[1:4])

9

In [90]:
# Typing 9 - Nominal Typing (as usual)
class Animal:
    pass

class Duck(Animal):
    def quack(self) -> None:
        print("Quack")

# Later add a penguin...
class Penguin(Animal):
    def quack(self) -> None:
        print("...Quark")

# Need to redefine make_it_quack to use Animal class
#  but... not every animal will have this method...
def make_it_quack(animal: Duck) -> None:
    animal.quack()

make_it_quack(Duck())
make_it_quack(Penguin())    # Works at runtime, but type checker (mypy) complains

Quack
...Quark


In [91]:
# Typing 10 - Structural Typing (describe capabilities, not ancestry/inheritence)
# C# this would be called interface!
# No Need to inherit from CanQuack!!!!

from typing_extensions import Protocol

class CanQuack(Protocol):
    def quack(self) -> None:
        ...

class Animal:
    pass

class Duck(Animal):
    def quack(self) -> None:
        print("Quack")

# Later add a penguin...
class Penguin(Animal):
    def quack(self) -> None:
        print("...Quark")

# make_it_quack uses CanQuack Interface
def make_it_quack(animal: CanQuack) -> None:
    animal.quack()

make_it_quack(Duck())
make_it_quack(Penguin())

Quack
...Quark
